In [1]:
#import os
import datajoint as dj

#os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz-12.2.1-win64\\bin' 

In [2]:
dj.config['database.host'] = "fyd2.nin.nl"
dj.config['database.user'] = "TM_user"
dj.config['database.password'] = "NExECh07pWp93bYg"
# dj.config.save_local()

In [3]:
schema = dj.schema('examplelab')

[2025-03-17 14:55:46,185][INFO]: Connecting TM_user@fyd2.nin.nl:3306
[2025-03-17 14:55:46,232][INFO]: Connected TM_user@fyd2.nin.nl:3306


In [4]:
@schema
class Subjects(dj.Manual):
    definition = '''
    subjectid       : varchar(50)   # unique user name
    ---
    species         : varchar(10) 
    projectidx      : int(3)
    idx             : int(5) auto_increment
    genotype = NULL : varchar(20)
    sex = 'U'       : enum('M', 'F', 'U')   #sex - Male, Female, or Unknown
    birthdate = NULL : date
    age = ''         : varchar(30)
    weight = ''      : varchar(20)
    hemisphere = 'u' : enum('l', 'r', 'u)    # left, right, uknown
    location = ''    : varchar(60)
    shortdescr = '' : varchar(250)
    UNIQUE INDEX (subjectid)
    INDEX (idx)
    '''

@schema
class Datasets(dj.Manual):
    definition = '''
    datasetid                   : varchar(50)                   # unique name
    -> Projects.proj(project="projectid")      
    ---
    projectidx                  : int(3)
    idx                         : int(3) auto_increment
    shortdescr = ''             : varchar(250)                  
    longdescr = ''              : varchar(6000)
    INDEX (idx)
    '''

@schema
class Projects(dj.Manual):
    definition = '''
    projectid                   : varchar(50)                   # unique name
    ---
    idx                         : int(3) auto_increment
    url = null                  : varchar(250)                  # 
    shortdescr = ''             : varchar(250)                  # 
    longdescr = ''              : varchar(20000)                 # 
    author = ''                 : varchar(100)
    entrydate = CURRENT_TIMESTAMP        : timestamp
    status = 0                           : tinyint(1)
    department_name = 'Example lab' : varchar(60)
    UNIQUE INDEX (idx)
    ''' 

    
@schema
class Conditions(dj.Manual):
    definition = '''
    conditionid          : varchar(30)                  
    dataset              : varchar(50)                  
    project              : varchar(60)                  
    ---
    idx                  : int auto_increment           
    shortdescr=null      : varchar(250)                 
    longdescr=null       : varchar(3000)                
    UNIQUE INDEX (idx)
    INDEX (dataset)
    ''' 

@schema
class Investigator(dj.Manual):
    definition = '''
    investigatorid       : varchar(30)                  
    ---
    idx                  : int auto_increment           
    email=null           : varchar(100)                 
    INDEX (idx)
    '''
    
@schema
class Scripts(dj.Manual):
    definition = '''
    scriptid             : varchar(50)                  
    idx                  : int auto_increment           
    ---
    url=null             : varchar(250)                 
    fileex=null          : varchar(5)                   
    shortdescr=null      : varchar(250)                 
    longdescr=null       : varchar(6000)                
    UNIQUE INDEX (scriptid)
    INDEX (idx)
    '''

    
@schema
class Setups(dj.Manual):
    definition = '''
    setupid              : varchar(30)                  
    ---
    url=null             : varchar(250)                 
    type=null            : enum('fMRI','ephys','ophys') 
    shortdescr=null      : varchar(250)                 
    longdescr=null       : varchar(6000)                
    UNIQUE INDEX (setupid)
        '''
    
@schema
class Stimulus(dj.Manual):
    definition = '''
    stimulusid           : varchar(50)                  
    ---
    idx                  : int auto_increment           
    url=null             : varchar(250)                 
    shortdescr=null      : varchar(250)                 
    longdescr=null       : varchar(6000)                
    UNIQUE INDEX (idx)
    '''
    
@schema
class ProjectsStimulus(dj.Manual):
    definition = '''
    -> Projects.proj(project="projectid")
    ---
    projectidx           : int                          
    -> Stimulus.proj(stimulus="stimulusid")
    stimulusidx          : int                          
    '''
    
@schema 
class ProjectsSubjects(dj.Manual):
    definition = '''
    -> Projects.proj(project="projectid")
    ---
    projectidx           : int                          
    -> Subjects.proj(subject="subjectid")
    subjectidx           : int  
    '''
    
@schema
class Sessions(dj.Manual):
    definition = '''
    idx                         : int(11) autoincrement   
    ---
    sessionid                   : varchar(100)                   # unique name
    url                         : varchar(500)                  # 
    server                      : varchar(30)                   # 
    -> Projects.proj(project="projectid")
    -> Datasets.proj(dataset="datasetid")
    -> Subjects.proj(subject="subjectid")
    -> Stimulus.proj(stimulus="stimulusid")
    -> Conditions.proj(excond="conditionid")
    -> Setups.proj(setup="setupid")
    date                        : date
    logfile                     : varchar(60)
    -> Investigator.proj(investid="investigatorid")
    UNIQUE INDEX (sessionid)
    INDEX (excond)
    '''



In [10]:
# dj.Diagram(schema)
# Subjects.describe()


In [6]:
#query = Sessions & 'project="aCCPainGain"' & 'subject="pg_tms_09"'
#query.fetch(as_dict=True)

In [7]:
Subjects()

subjectid,species,projectidx,idx,genotype,sex,birthdate,age,weight,hemisphere,location,shortdescr
20231113,mouse,66,30,C mouse,M,None,,,u,,
bob1,mouse,1,37,Wildtype,M,2025-03-17,,15gram,u,visual cortex,another mouse
bobby,mouse,63,29,,U,None,,,u,,sdfs
boef,monkey,None,24,none,M,None,,,u,,
D115,mouse,63,23,none,M,None,,,u,,Experimental group D
goofy,monkey,None,25,none,M,2016-05-11,,,r,V2,Trained on figure ground task
hoby,mouse,None,27,none,M,None,,,u,,
maybe,mouse,None,26,something,F,2024-05-30,,,u,,maybe this mouse
mickey,mouse,None,28,Ach_Cre,M,2021-02-19,,,l,visual cortex,this is a mouse
